In [3]:
import os
import datetime
import time
import cv2
import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'cv2'

In [6]:
# Path to frozen detection graph
EXPORT_NAME = 'ssd_mobilenetv2_1_224_800000'
MODEL_NAME = 'ssd_mobilenetv2_1_224'
CWD_PATH = os.getcwd()
PATH_TO_CKPT = os.path.join(CWD_PATH, '..', 'training/models', MODEL_NAME, 'export', EXPORT_NAME, 'frozen_inference_graph.pb')

In [7]:
g = tf.Graph()
run_meta = tf.RunMetadata()
with g.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    
    opts = tf.profiler.ProfileOptionBuilder.float_operation() 
    flops = tf.profiler.profile(g, run_meta=run_meta, cmd='op', options=opts)
    #flops = tf.profiler.profile(g, options=tf.profiler.ProfileOptionBuilder.float_operation())
    #print('FLOP = ', flops.total_float_ops)
    if flops is not None:
        print('Total FLOPs:',flops.total_float_ops)

Total FLOPs: 8457


In [40]:
import numpy as np

ckpt_fpath = os.path.join(CWD_PATH, '..', 'training/models', MODEL_NAME, 'export', EXPORT_NAME, 'model.ckpt')
# Open TensorFlow ckpt
reader = tf.train.NewCheckpointReader(ckpt_fpath)

print('\nCount the number of parameters in ckpt file(%s)' % ckpt_fpath)
param_map = reader.get_variable_to_shape_map()
total_count = 0
for k, v in param_map.items():
    if 'Momentum' not in k and 'global_step' not in k:
        temp = np.prod(v)
        total_count += temp
        #print('%s: %s => %d' % (k, str(v), temp))

print('Total Param Count: %d' % total_count)


Count the number of parameters in ckpt file(/Users/lindawang/OLIV/eval/../training/models/ssd_mobilenetv2_1_300/export/ssd_mobilenetv2_1_300_800000/model.ckpt)
Total Param Count: 4707674
